In [1]:
import re
import numpy as np
from input import *

In [2]:
#damit das funktioniert müssten erst alle Typen und Funktionsnamen in Strings umgewandelt werden
#https://stackoverflow.com/questions/3061/calling-a-function-of-a-module-by-using-its-name-a-string

#import json
#
#with open('keywords.json', 'w') as f:
#    f.writelines(json.dumps(keywords))  

In [ ]:
k_mod = {
    '1': {
        'PERM': 0.6,
        'LONG': 0.7,
        'MEDI': 0.8,
        'SHOR': 0.9,
        'MSHO': 1.0,
        'VSHO': 1.1,
    },
    '2': {
        'PERM': 0.6,
        'LONG': 0.7,
        'MEDI': 0.8,
        'SHOR': 0.9,
        'MSHO': 1.0,
        'VSHO': 1.1,
    },
    '3': {
        'PERM': 0.5,
        'LONG': 0.55,
        'MEDI': 0.65,
        'SHOR': 0.7,
        'MSHO': 0.8,
        'VSHO': 0.9,
    },
}
#(PERM, LONG, MEDI, SHOR, MSHO, VSHO)

# Programm


In [3]:
def parse_input_file(file: str, modules: str):
    module_names = ['MATQS', 'SYSTEM', 'LOAD', 'CALC', 'COMB']

    m_prog = re.compile(r'[\+|\-]?PROG', re.I)  # Initialisiere Regex-Muster
    m_ende = re.compile(r'ENDE', re.I)

    module_nr = None                    # Prüfe, ob die Angabe der zu berechnenden Module eine Zahl ist
    if modules.isnumeric():
        module_nr = int(modules)

    with open(file) as f:
        prog_counter = 0
        current_prog_name, current_module_name = "", ""
        save_lines = False
        end_programm = False
        lines = {}
        db = {}

        for i, line in enumerate(f.readlines()):
            line_nr = i + 1

            line = line.strip()     #TODO Kommentare hinter der Zeile auch gleich entfernen
            if len(line) == 0 or line[0] == '$':        # leere Zeilen oder solche mit Kommentar direkt überspringen
                    continue
            
            # Suche Ende eines Moduls
            if (s_ende := m_ende.match(line)):          # wenn Modul beendet: Werte zurücksetzen und parsen einleiten
                if lines:                               # parsen nur, wenn Zeilen enthalten
                    module_db = parse_module(db, current_module_name, lines) 
                    db[list(module_db.keys())[0]] = module_db[list(module_db.keys())[0]] #TODO Beschreibung         
                lines = {}
                save_lines = False

                if end_programm:        # Abbruch, falls nur ein bestimmtes Modul zu berechnen
                    break

            # Zeilen in Modul
            if save_lines:                          # falls in gesuchten Modul: Zeile speichern
                lines[line_nr] = line.strip()

            # suche Start eines Moduls
            if (s_prog_list:= list(m_prog.finditer(line))): # suche Anfang eines Moduls
                s_prog = s_prog_list[0]
                if '$' in line[:s_prog.span()[0]]:          # falls Kommentar vor "prog": ignorieren
                    continue

                for next_prog in s_prog_list[1:]:           # falls mehrere "prog": prüfe ob alle nach Kommentar, sonst Warnung
                    if '$' in line[:next_prog.span()[0]]:
                        break
                    else:
                        raise Warning(f"Multiple Occurrences of PROG in line {line_nr}.")

                if save_lines:                          # falls schon aktives Modul: warnen
                    raise Warning(f"Nested Modules in line {line_nr}.")

                # falls "prog" in Zeile nach anderem Zeichen, aber vor Kommentar: Warnung    
                if s_prog.span()[0] != 0:          
                    raise SyntaxWarning(f"Unrecognized Character before PROG at line {line_nr}.")
                else:
                    prog_counter += 1   # erhöhe Anzahl gefundener Module

                    current_prog_name = line[s_prog.span()[1]:].strip().split(" ")[0]
                    current_module_name = current_prog_name.split("_")[0].upper()
                    
                    if current_module_name not in module_names:             # Prüfe ob Modulname bekannt
                        raise Exception(f'Module name: "{current_module_name}" unknown in line {line_nr}.')

                    if modules == "":                                       # alle Module sollen berechnet werden
                        save_lines = True
                    elif line[s_prog.span()[0]] != '-' and modules == '+':  # nur Module ohne Minus sollen berechnet werden
                        save_lines = True
                    elif current_prog_name.upper() == modules.upper():      # nur Module mit bestimmten Namen sollen berechnet werden
                        save_lines = True
                        end_programm = True                 #TODO drinlassen? -> berechnet nur erstes mit Modul-Name
                    elif prog_counter == module_nr:                         # nur x-tes Modul des Programm berechnen
                        save_lines = True
                        end_programm = True

    return db                 

In [5]:
#TODO falls Sternchen statt Wert für key -> nimm default
#TODO Exzentrizität für Querschnitte (YM, ZM)?
keywords = {
    'MATQS': {
        'MAT': {
            'KEYS': {       #TODO STAH, HOLZ definieren und Werte in Datenbank hinterlegen
                'NR':   (int,   None, False),       # Materialnummer
                'E':    (float, 210000, False),     # E-Modul, standardmäßig für Baustahl [MN/m²]
                'GAM':  (float, 78.5, False),       # Wichte, standardmäßig für Baustahl [kN/m³]
                'ALFA': (float, 12  , False),       # Temperaturausdehnungskoeffizient alpha [1/K *E-5]
                'BEZ':  (str,   ""  , False),       # Bezeichnung Material
            },
            'FUNC': create_MAT,
        },
        'QUER': {
            'KEYS': {       
                'NR':   (int,   None, False),       # Querschnittsnummer
                'MNR':  (int,   1,    False),       # Materialnummer
                'A':    (float, 1,    False),       # Querschnittsfläche [m²]
                'IY':   (float, None, False),       # Flächenträgheitsmoment [m^4]; Standard = A^3/12
                'BEZ':  (str,   ""  , False),       # Bezeichnung Querschnitt
            },
            'FUNC': create_CS_QUER,
        },
        'QB': {     #TODO Rotation
            'KEYS': {       
                'NR':   (int,   None, False),       # Querschnittsnummer
                'MNR':  (int,   1,    False),       # Materialnummer
                'H':    (float, 1,    False),       # Querschnittshöhe [m]
                'B':    (float, 1,    False),       # Querschnittsbreite [m]
                'BEZ':  (str,   ""  , False),       # Bezeichnung Querschnitt
            },
            'FUNC': create_CS_QB,
        },
        'QC': {
            'KEYS': {       
                'NR':   (int,   None, False),       # Querschnittsnummer
                'MNR':  (int,   1,    False),       # Materialnummer
                'D':    (float, None, True ),       # Durchmesser [m]
                'T':    (float, 0,    False),       # Wandstärke [m]; falls T=0 -> Vollkreis
                'BEZ':  (str,   ""  , False),       # Bezeichnung Querschnitt
            },
            'FUNC': create_CS_QC,
        },
        'PROF': {   #TODO nicht funktionsfähig -> Profilliste erstellen #TODO Rotation
            'KEYS': {       
                'NR':   (int,   None, False),       # Querschnittsnummer
                'MNR':  (int,   1,    False),       # Materialnummer
                'TYP':  (float, None, True ),       # Profiltyp
                'Z1':   (float, None, True ),       # Bezeichner Profil
                'Z2':   (float, None, False),       # Bezeichner Profil
                'BEZ':  (str,   ""  , False),       # Bezeichnung Querschnitt
            },
            'FUNC': "",
        },
        #'QNR': {},
        #'QW': {},
    },
    'SYSTEM': {
        'STEU': {
            'KEYS': {
                'HMIN': (float, -10 , False),       # globaler Stabteiler: positiv -> max. Abstand Zwischenknoten [m], negativ + int -> Anzahl Unterteilungen
            },
            'FUNC': change_SYSTEM_GLOBALS,
        },
        'SPT': {
            'KEYS': {
                'NR':   (int,   None, False),       # Knotennummer
                'X':    (float, 0   , False),       # x-Koordinate [m]
                'Z':    (float, 0   , False),       # z-Koordinate [m]
                'FIX':  (str,   ""  , False),       # Auflagerbedingungen (PX, PZ, PP, MY, FF)
                'NX':   (float, 1   , False),       # Ausrichtung lokales Koordinatensystem
                'NZ':   (float, 0   , False),   
                'ALPH': (float, 0   , False),       # entweder über Drehwinkel oder Vorgabe Richtung [°]
                'BEZ':  (str,   ""  , False),       # Bezeichnung Knoten
            },
            'FUNC': create_SPT,
        },  
        'SPTS': {  
            'KEYS': {
                'CP':   (float, 0   , False),       # Federsteifigkeit in axiale Richtung oder in lokal x [kN/m]
                'CQ':   (float, 0   , False),       # Federsteifigkeit in Querrichtung oder in lokal z [kN/m]
                'CM':   (float, 0   , False),       # Rotationssteifigkeit [kNm/rad]
            },
            'FUNC': add_SPTS,
        },  
        'SLN': {  
            'KEYS': {
                'NR':   (int,   None, False),       # Stabnummer
                'NPA':  (int,   None, True ),       # Knoten am Stabanfang
                'NPE':  (int,   None, True ),       # Knoten am Stabende
                'SDIV': (float, None, False),       # lokaler Stabteiler: positiv -> max. Abstand Zwischenknoten [m], negativ + int -> Anzahl Unterteilungen; überschreibt STEU HMIN
                'QNR':  (int,   1,    False),       # Querschnittsnummer
                'FIXA': (str,   None, False),       # Gelenkbedingung am Stabanfang (N, V oder VZ, PP, M oder MY oder MM)
                'FIXE': (str,   None, False),       # Gelenkbedingung am Stabende (N, V oder VZ, PP, M oder MY oder MM)
                'BEZ':  (str,   ""  , False),       # Bezeichnung Stab
            },
            'FUNC': create_SLN,  
        },
    },
    'LOAD': {
        'LF': {},
        'KNOT': {},
        'STAB': {},
        'STEL': {},
        'COPY': {},
    }
}

In [6]:
def parse_module(db: dict, module_name: str, lines: dict) -> None:
    m_int = re.compile(r'^\-?\d+$')                 # prüfe ob String ein Integer ist
    m_float = re.compile(r'^\-?\d+(?:\.\d+)?$')     # prüfe ob String ein Float ist
    #TODO Exponentialschreibweise ermöglichen?

    #TODO je nach Modul fortgeschrittenere Einträge löschen -> in parse_input erledigen

    if module_name == 'MATQS':
        module_db = initiate_matqs()
    elif module_name == 'SYSTEM':
        module_db = initiate_system()
    elif module_name == 'LOAD':
        module_db = initiate_load()
    elif module_name == 'CALC':
        module_db = initiate_calc()
    elif module_name == 'COMB':
        module_db = initiate_comb()

    #print(db)

    for line_nr, line in lines.items():
        try:
            words = split_no_string(line)
        except:
            raise Exception(f'Closing parenthesis missing in line {line_nr}.')
        #TODO Warnung wenn Zeile nicht komplett verstanden
        if (command := words[0]) not in keywords[module_name]:
            raise Exception(f'Command: "{command}" unknown at position 1 in line {line_nr}.')
            
        comm_words = keywords[module_name][command.upper()]['KEYS']
        
        line_dict = {}
        current_key = words[1]
        current_value = ""

        for word in words[2:]: 
            if word.upper() in comm_words.keys():
                line_dict[current_key] = current_value
                current_key = word.upper()
                current_value = ""
            else:
                current_value += word

        if current_value == "":
            raise Exception(f'Last key is missing a value in line {line_nr}.')   
        else: 
            line_dict[current_key] = current_value
        #TODO prüfe ob alle benötigten Angaben vorhanden -> alle benötigten keys vor Schleife zusammensuchen
        #TODO SPTS darf nur direkt hinter SPT stehen -> wie ist das erreichbar

        for key, value in line_dict.items():
            reqs = comm_words[key]

            if check_formula(value):
                try:
                    res = eval(value)
                    value = str(res)
                except:
                    raise Exception(f'Could not evaluate formula for key: {key} in line {line_nr}.')

            if reqs[0] == int:
                if not m_int.match(value):
                    raise Exception(f'Expected Integer for key: {key} in line {line_nr}.')
                else:
                    line_dict[key] = int(value)
            elif reqs[0] == float:
                if not m_float.match(value):
                    raise Exception(f'Expected Float for key: {key} in line {line_nr}.')
                else:
                    line_dict[key] = float(value)
            elif reqs[0] == str:
                line_dict[key] = value.strip('"\'')

        #TODO erzeuge DB
        try:
            update_db = keywords[module_name][command.upper()]['FUNC'](db, module_db, line_dict, comm_words)
        except Exception as inst:
            if len(inst.args) >= 1:
                inst.args = (inst.args[0] + f' (line {line_nr}).',) + inst.args[1:]
            raise inst
            #raise type(inst)(str(inst) + f' in line {line_nr}. ')
        print(command, update_db)
        #print(line_dict)
        if update_db == None:       #TODO entfernen, nur zur Kontrolle
            continue
        
        for key, value in update_db.items():
            module_db[module_name.lower()][key].update(value)

    return module_db

parse_input_file('input.dat', 'system')

STEU {'globals': {'HMIN': -20.0}}
SPT {'nodes': {'1': {'x': 0.0, 'z': 0.0, 'sup': {'x': -1, 'z': -1, 'y': -1}, 'alpha': 0, 'bez': 'Auflager A'}}}
SPT {'nodes': {'2': {'x': 6.0, 'z': 0.0, 'sup': {'x': 0, 'z': 0, 'y': 0}, 'alpha': 0, 'bez': 'Zwischenknoten'}}}
SPT {'nodes': {'3': {'x': 12.0, 'z': 0.0, 'sup': {'x': 0, 'z': 0, 'y': 0}, 'alpha': 0, 'bez': 'Auflager B'}}}
SPTS {'nodes': {'3': {'x': 12.0, 'z': 0.0, 'sup': {'x': 0, 'z': 0, 'y': 1000.0}, 'alpha': 0, 'bez': 'Auflager B'}}}
SPT {'nodes': {'4': {'x': 12.0, 'z': 6.0, 'sup': {'x': -1, 'z': -1, 'y': 0}, 'alpha': 0, 'bez': 'Auflager C'}}}
SPT {'nodes': {'1': {'x': 3.0, 'z': 0.0, 'sup': {'x': -1, 'z': -1, 'y': -1}, 'alpha': 0, 'bez': 'Test'}}}


Exception: No materials or cross sections defined (line 21).

In [7]:
def change_MATQS_globals():
    pass

#TODO falls Sternchen statt Wert für key -> nimm default
#TODO Exzentrizität für Querschnitte (YM, ZM)?
keywords = {
    'MATQS': {
        'STEU': {
            'KEYS': {
                'NKL':  (int,   1,    False),       # Nutzungsklasse (1, 2, 3)
            },
            'FUNC': change_MATQS_globals,
        },
        'MAT': {
            'KEYS': {       #TODO STAH, HOLZ definieren und Werte in Datenbank hinterlegen
                'NR':   (int,   None, False),       # Materialnummer
                'E':    (float, 210000, False),     # E-Modul, standardmäßig für Baustahl [MN/m²]
                'GAM':  (float, 78.5, False),       # Wichte, standardmäßig für Baustahl [kN/m³]
                'ALFA': (float, 12  , False),       # Temperaturausdehnungskoeffizient alpha [1/K *E-5]
                'BEZ':  (str,   ""  , False),       # Bezeichnung Material
            },
            'FUNC': create_MAT,
        },
        'QUER': {
            'KEYS': {       
                'NR':   (int,   None, False),       # Querschnittsnummer
                'MNR':  (int,   1,    False),       # Materialnummer
                'A':    (float, 1,    False),       # Querschnittsfläche [m²]
                'IY':   (float, None, False),       # Flächenträgheitsmoment [m^4]; Standard = A^3/12
                'BEZ':  (str,   ""  , False),       # Bezeichnung Querschnitt
            },
            'FUNC': create_CS_QUER,
        },
        'QB': {     #TODO Rotation
            'KEYS': {       
                'NR':   (int,   None, False),       # Querschnittsnummer
                'MNR':  (int,   1,    False),       # Materialnummer
                'H':    (float, 1,    False),       # Querschnittshöhe [m]
                'B':    (float, 1,    False),       # Querschnittsbreite [m]
                'BEZ':  (str,   ""  , False),       # Bezeichnung Querschnitt
            },
            'FUNC': create_CS_QB,
        },
        'QC': {
            'KEYS': {       
                'NR':   (int,   None, False),       # Querschnittsnummer
                'MNR':  (int,   1,    False),       # Materialnummer
                'D':    (float, None, True ),       # Durchmesser [m]
                'T':    (float, 0,    False),       # Wandstärke [m]; falls T=0 -> Vollkreis
                'BEZ':  (str,   ""  , False),       # Bezeichnung Querschnitt
            },
            'FUNC': create_CS_QC,
        },
        'PROF': {   #TODO nicht funktionsfähig -> Profilliste erstellen #TODO Rotation
            'KEYS': {       
                'NR':   (int,   None, False),       # Querschnittsnummer
                'MNR':  (int,   1,    False),       # Materialnummer
                'TYP':  (float, None, True ),       # Profiltyp
                'Z1':   (float, None, True ),       # Bezeichner Profil
                'Z2':   (float, None, False),       # Bezeichner Profil
                'BEZ':  (str,   ""  , False),       # Bezeichnung Querschnitt
            },
            'FUNC': "",
        },
        #'QNR': {},
        #'QW': {},
    },
    'SYSTEM': {
        'STEU': {
            'KEYS': {#TODO Eigengewichtsrichtung
                'HMIN': (float, -10,  False),       # globaler Stabteiler: positiv -> max. Abstand Zwischenknoten [m], negativ + int -> Anzahl Unterteilungen
            },
            'FUNC': change_SYSTEM_GLOBALS,
        },
        'SPT': {
            'KEYS': {
                'NR':   (int,   None, False),       # Knotennummer
                'X':    (float, 0   , False),       # x-Koordinate [m]
                'Z':    (float, 0   , False),       # z-Koordinate [m]
                'FIX':  (str,   ""  , False),       # Auflagerbedingungen (PX, PZ, PP, MY, FF)
                'NX':   (float, 1   , False),       # Ausrichtung lokales Koordinatensystem
                'NZ':   (float, 0   , False),   
                'ALPH': (float, 0   , False),       # entweder über Drehwinkel oder Vorgabe Richtung [°]
                'BEZ':  (str,   ""  , False),       # Bezeichnung Knoten
            },
            'FUNC': create_SPT,
        },  
        'SPTS': {  
            'KEYS': {
                'CP':   (float, 0   , False),       # Federsteifigkeit in axiale Richtung oder in lokal x [kN/m]
                'CQ':   (float, 0   , False),       # Federsteifigkeit in Querrichtung oder in lokal z [kN/m]
                'CM':   (float, 0   , False),       # Rotationssteifigkeit [kNm/rad]
            },
            'FUNC': add_SPTS,
        },  
        'SLN': {  
            'KEYS': {
                'NR':   (int,   None, False),       # Stabnummer
                'NPA':  (int,   None, True ),       # Knoten am Stabanfang
                'NPE':  (int,   None, True ),       # Knoten am Stabende
                'SDIV': (float, None, False),       # lokaler Stabteiler: positiv -> max. Abstand Zwischenknoten [m], negativ + int -> Anzahl Unterteilungen; überschreibt STEU HMIN
                'QNR':  (int,   1,    False),       # Querschnittsnummer
                'FIXA': (str,   ""  , False),       # Gelenkbedingung am Stabanfang (N, V oder VZ, PP, M oder MY oder MM)
                'FIXE': (str,   ""  , False),       # Gelenkbedingung am Stabende (N, V oder VZ, PP, M oder MY oder MM)
                'BEZ':  (str,   ""  , False),       # Bezeichnung Stab
            },
            'FUNC': create_SLN,  
        },
    },
    'LOAD': {
        'ACT': {
            'KEYS': {
                'TYP':  (str,   None, True ),       # Bezeichnung
                'GAMU': (float, 1.35, False),       # Teilsicherheitsbeiwert ungünstig
                'GAMF': (float, 1.0,  False),       # Teilsicherheitsbeiwert günstig
                'PSI0': (float, 1.0,  False),       # Kombinationsbeiwert selten
                'PSI1': (float, 1.0,  False),       # Kombinationsbeiwert häufig
                'PSI2': (float, 1.0,  False),       # Kombinationsbeiwert quasi-ständig
                #'KMOD': (float, 1.0,  False),       # Modifikationsbeiwert (für den Holzbau)
                'KLED': (str,   'PERM', False),     # Klasse der Lasteinwirkungsdauer (PERM, LONG, MEDI, SHOR, MSHO, VSHO)
                'PART': (str,   'G',  False),       # Art der Einwirkung (G oder Q)
                #'SUP':  (str,   'PERM', True),      # Standardüberlagerung siehe sofiload S. 22
                'BEZ':  (str,   ""  , False),       # Bezeichnung Einwirkung
            },
            'FUNC': "",
        },
        'LF': {
            'KEYS': {
                'NR':   (int,   None, False),       # Lastfallnummer
                'TYP':  (str,   None, True),        # Typ des Lastfalls (definiert in ACT)
                'FAKT': (float, 1.0,  False),       # Faktor Lastfall (wirkt nicht auf FAKG)
                'FAKG': (float, 0.0,  False),       # Faktor Eigengewicht (keine anderen Lasten mehr ansetzbar) #TODO was ist mit EGZ?
                'BEZ':  (str,   ""  , False),       # Bezeichnung Lastfall
            },
            'FUNC': "",
        },
        'KNOT': {#TODO falls EXX oder EZZ 0 -> kein Moment erzeugen (zweimal X -> global, einmal lokal)
            'KEYS': {
                'NR':   (int,   '1' , False),       # Knotennummer
                'TYP':  (str,   'PG', False),       # Lastart und Richtung (PG, PX(X), PZ(Z), MY(Y), P(P), V(V), WX(X), WZ(Z), DY(Y))
                'P1':   (float, 0,    False),       # Lastwert
                'EXX':  (float, 0,    False),       # Exzentrizität in x-Richtung [m]
                'EZZ':  (float, 0,    False),       # Exzentrizität in x-Richtung [m]
            },
            'FUNC': "",
        },
        'STAB': {#TODO Rechne DT und DTZ intern in EX, KY um
            'KEYS': {
                'NR':   (int,   '1' , False),       # Stabnummer
                'TYP':  (str,   'PG', False),       # Lastart und Richtung (PG, PX(X), PZ(Z), EX, KY, DT, DTZ)
                'PA':   (float, 0,    False),       # Lastwert Anfang
                'PE':   (float, None, False),       # Lastwert Ende (default wie Anfang)
                'A':    (float, 0,    False),       # Abstand des Lastanfangs vom Stabanfang/-ende [m]
                'L':    (float, None, False),       # Länge der Last [m]
                'MESS': (str,   'S',  False),       # Bezugssystem für A, L (S, XX, ZZ) (- voranstellen, falls auf Stabende bezogen)
            },
            'FUNC': "",
        },
        'STEL': {
            'KEYS': {
                'NR':   (int,   '1' , False),       # Stabnummer
                'TYP':  (str,   'PG', False),       # Lastart und Richtung (PG, PX(X), PZ(Z), MY(Y))
                'P':    (float, 0,    False),       # Lastwert Anfang
                'A':    (float, 0,    False),       # Abstand der Last vom Stabanfang/-ende [m]
                'MESS': (str,   'S',  False),       # Bezugssystem für A, L (S, XX, ZZ) (- voranstellen, falls auf Stabende bezogen)
            },
            'FUNC': "",
        },
        'COPY': {
            'KEYS': {
                'NR':   (int,   None, True ),       # Stabnummer
                'FAKT': (float, 1.0,  False),       # Faktor Lastfall
            },
            'FUNC': "",
        },
    }, 
    'COMB': {# Lastfälle DESI, RARE, FREQ, PERM; #TODO ACT benötigt?
        'STEU': {
            'KEYS': {
                'KMOD': (bool,  'Nein', False),     # Steuert Überlagerung getrennt nach k_mod-Werten
            },
            'FUNC': "",
        },
        'KOMB': {
            'KEYS': {
                'NR':   (int,   '1',  False),       # Nummer der Kombination
                'EXTR': (str,   'STAN', False),     # Überlagerungsart (STAN, DESI, PERM, RARE, FREQ)
                'BASE': (int,   None, False),       # Basislastfallnummer (Vielfaches von 100)
            },
            'FUNC': "",
        },
        'LF': {
            'KEYS': {
                'NR':   (int,   None, True ),       # Nummer der Lastfalls
                'TYPE': (str,   None, True ),       # Überlagerungsart (G (PERM), PERC, Q (COND), AG_ (ALEX=AG1), A_ (EXCL))
            },
            'FUNC': "",
        },
        'SUPP': {#KNOT: PX, PZ, MY, UX, UZ, URY, global und lokal? #STAB N, VZ, MY, UX, UZ, URY? #FEDE P, PT, M
            'KEYS': {
                'KOMB': (int,   '1',  False),       # Nummer der Kombination
                'EXTR': (str,   'MAMI', False),     # Extremalbedingung (MAX, MIN, MAMI)
                'ETYP': (str,   None, True ),       # Elementtyp (KNOT, STAB, FEDE?)
                'BEZ':  (str,   ""  , False),       # Bezeichnung Überlagerungslastfall
            },
            'FUNC': "",
        },
    }
}


#print(parse_input_file('input.dat', '+'))#['system'])